In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from Deep_Learning.Experimental.Boltzmann.train import train, eval
from Deep_Learning.Experimental.Boltzmann.model import FCBoltzmannModel
from Utils.dataset import PreloadedDataset

In [2]:
eval_criterion=nn.MSELoss()
model = FCBoltzmannModel(2, 2, [1])
step = 0

In [3]:
def dprint(model, state):
    print(f'state: {state}')
    for i, layer in enumerate(model.layers):
        print(f'layer {i}:\nweight: {layer.weight.data}\nbias: {layer.bias.data}\n')


In [4]:
state = [torch.tensor([[1.0, 1.0]]), torch.tensor([[1.0]]), torch.tensor([[0.0, 1.0]])]
dprint(model, state)

state: [tensor([[1., 1.]]), tensor([[1.]]), tensor([[0., 1.]])]
layer 0:
weight: tensor([[0.5742, 0.2785]])
bias: tensor([-0.0741])

layer 1:
weight: tensor([[-0.8293],
        [ 0.2271]])
bias: tensor([0.4036, 0.3866])



In [5]:
updateds = 0
nums = 10000
layer_i = 2
node_i = 0
temperature = 1.0
for i in range(nums):
    energy_gap = model._energy_gap(state, layer_i, node_i)
    p_1 = 1.0 / (1 + (-energy_gap/temperature).exp())
    activation = (p_1 > torch.rand(p_1.shape)).float()
    updated = not torch.equal(state[layer_i][:,node_i], activation)
    updateds += updated
    if i == 0:
        print(f'energy_gap: {energy_gap.item():.3f}')
        print(f'p_1: {p_1.item():.3f}')
print(updateds/nums)


energy_gap: -1.059
p_1: 0.257
0.2578


In [6]:
updateds = 0
nums = 1
rands = []
for i in range(nums):
    state = [torch.tensor([[1.0, 1.0]]), torch.tensor([[1.0]]), torch.tensor([[0.0, 1.0]])]
    if i == 1000:
        debug = True
    else:
        debug = False
    updated = model._update_node(state, layer_i, node_i, temperature = 1.0, debug=debug)
   
    # energy_gap = model._energy_gap(state, layer_i, node_i)
    # p_1 = 1.0 / (1 + (-energy_gap/temperature).exp())
    # rand = torch.rand(p_1.shape)
    # activation = (p_1 > rand).float()
    # updated = not torch.equal(state[layer_i][:,node_i], activation)
    # if debug:
    #     print(f'energy_gap: {energy_gap}')
    #     print(f'p_1: {p_1}')
    #     print(f'rand: {rand}')
    #     print(f'activation: {activation}')


    updateds += updated

print(updateds/nums)
state

0.0


[tensor([[1., 1.]]), tensor([[1.]]), tensor([[0., 1.]])]

In [21]:
model = FCBoltzmannModel(728, 32, [256])
x = torch.randint(0, 1, (1, 728)).float()
for _ in range(5):
    out = model(x, max_steps=20, temp_coeffs=(2.0, 2.0))